In [ ]:
import requests
import json

def TransformQueryText_Product(query_text):
    #Format query text for product search
    #Expected format is user search text -> user search text OR user~2 search~2 text~2
    base_text = query_text.replace("'", "\'").replace("/", "\/").replace("-","").replace('"', '')
    query_parts = base_text.split(' ')
    formatted_query_text = ''
    for part in query_parts:
        formatted_query_text += "{0} ".format(part)
    formatted_query_text += "OR "
    for part in query_parts:
        formatted_query_text += '{0}~2 '.format(part)
    return formatted_query_text

def TransformQueryText_Part(query_text):
    #Format query text for part search
    #Expected format is user search text -> user^6 search^6 text^6 OR user~2 search~2 text~2
    base_text = query_text.replace("'", "\'").replace("/", "\/").replace("-","").replace('"', '')
    query_parts = base_text.split(' ')
    formatted_query_text = ''
    for part in query_parts:
        formatted_query_text += "{0}^6 ".format(part)
    formatted_query_text += "OR "
    for part in query_parts:
        formatted_query_text += '{0}~2 '.format(part)
    return formatted_query_text

def ExecuteQuery(fuzzy_text, product_search=True):
    #Submits a query to azure search (routing to product or part) and returns results
    body = {
        "api-version": "2019-05-06",
        "search": fuzzy_text.strip(),
        "queryType": "full",
        "searchFields": "product_name",
        "highlight": "product_name",
    }
    if not product_search:
        body["searchFields"] = "description"
        body["highlight"] = "description"
    headers = {
        "Content-Type": "application/json",
        "api-key": "AZURE_SEARCH_KEY"
    }
    base_url = 'BASE_URL1'
    if not product_search:
        base_url = 'BASE_URL2'

    r = requests.get(base_url, params=(body), headers=headers)
    results = (r.json())
    hits = results['value']
    
    all_hits = hits
    all_hits.sort(key=lambda x: x['@search.score'], reverse=True)
    
    return_hits = {}
    index = 0
    while len(return_hits.keys())<10 and index < len(all_hits):
        if product_search:
            return_hits[all_hits[index]['product_id']] = all_hits[index]
        else:
            return_hits[all_hits[index]['part_number']] = all_hits[index]
        index += 1
        
    return_list = list(return_hits.values())
    return_list.sort(key=lambda x: x['@search.score'], reverse=True)
    return return_list

def LUIS_Request(user_text):
    #TO-DO: submit user text to LUIS model to retrieve simple entities and intent
    pass
    
def LUIS_Parse(luis_result):
    #TO-DO: pull out simple entities and intent from LUIS results
    pass
    
def CompareProductPartSearchResults(product_results, part_results):
    #Compares product and part search results
    
    #If product results are present and no part results exist, product wins
    if len(product_results)>0 and len(part_results)==0:
        return 'product'
    #If part results are present and no product results exist, part wins
    elif len(product_results)==0 and len(part_results)>0:
        return 'part'
    #Otherwise, compare the magnitude of the first product and part result. If part is greater it wins, and if product is greater it wins.
    else:
        if product_results[0]['@search.score']>part_results[0]['@search.score']:
            return 'product'
        else:
            return 'part'
        
def PrintResults(product_results, part_results, result_type='product'):
    #Helper function to print winning index and top 10 results
    if result_type=='product':
        print('Products')
        print()
        for result in product_results:
            print(result['product_name'] + ' ' + str(result['@search.score']))
    else:
        print('Parts')
        print()
        for result in part_results:
            print(result['description'] + ' ' + str(result['@search.score']))
    
    
def Unified_Request(user_text):
    #1. Submit full user text to LUIS for intent/simple entity extraction
    
    #2. Parse LUIS results and dispatch to appropriate skill (assuming search OR none here)
    
    #3. Query and store product search results
    product_results = ExecuteQuery(TransformQueryText_Product(user_text), True)
    
    #4. Query and store part search results
    part_results = ExecuteQuery(TransformQueryText_Part(user_text), False)
    
    #5. Identify strongest scoring top result
    best_index = CompareProductPartSearchResults(product_results, part_results)
    
    #6. Print best results
    PrintResults(product_results, part_results, best_index)

In [ ]:
#Enter full user utterance here
USER_TEXT = 'lookup sample product/part name'
        
Unified_Request(USER_TEXT)